# 05 — Portfolio Analytics (Vectorization, KRD, Credit Spread, Combined Grid)

## Objective
Demonstrate how this scales beyond single-name pricing:
- vectorized pricing across a portfolio (cashflow expansion + discount once per date)
- portfolio DV01 / convexity
- key rate duration (bucketed DV01 by curve knots)
- corporate spread pricing + spread DV01
- combined rate/spread scenario grid

This notebook depicts the following workflow:
- compute market value
- compute rate risk
- compute spread risk
- explain scenario P&L

In [ ]:
import numpy as np
import pandas as pd

from fixed_income_engine.curves import bootstrap_curve_from_bills_notes
from fixed_income_engine.portfolio import (
    make_sample_portfolio,
    price_portfolio_vectorized,
    add_random_spreads,
    price_corporate_portfolio_vectorized,
)
from fixed_income_engine.risk import (
    compute_portfolio_dv01,
    compute_krd_hat,
    portfolio_spread_dv01,
)
from fixed_income_engine.scenarios import (
    run_combined_rate_spread_scenarios,
)

val_date = pd.Timestamp("2026-02-13")
settle = pd.Timestamp("2026-02-16")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

curve = bootstrap_curve_from_bills_notes(market, val_date)

portfolio_df = make_sample_portfolio(n=20, val_date=val_date, seed=7)